<a href="https://colab.research.google.com/github/simonfelding/reddit-citations-to-RIS-format/blob/main/reddit_to_bibtex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install python-docx 

In [9]:
document = "speciale (1).docx" # finds reddit.com urls in a word document. Must look like this: (reddit.com/r/subform/)

In [10]:
# convert reddit url to RIS file
import json
import urllib
from urllib.request import urlopen
from datetime import datetime
from time import sleep
from tqdm.auto import tqdm
from docx import Document
import re

document = Document(document)
urls = []
for paragraph in document.paragraphs:
  text=re.findall('\((.*?)\)',paragraph.text)
  for word in text:
    if "reddit.com/r/" in word:
      urls.append(word)
print(urls)


def url_to_api(url):
  url = url.split("reddit.com/r/")[1]
  parts = url.split('/') #0 subreddit # 1 comments #2 uuid #3 title
  return f"https://api.reddit.com/api/info/?id=t3_{parts[2]}&raw_json=1"

['reddit.com/r/BPD/comments/7x3jv4/i_feel_like_the_universe_is_asking_me_to_die/', 'reddit.com/r/BPD/comments/akhm9o/paranoia/', 'reddit.com/r/BPD/comments/9o804o/just_looking_for_some_advice_with_paranoia/', 'reddit.com/r/BPD/comments/4z7bks/in_a_new_relationship/', 'reddit.com/r/BPD/comments/68ighc/feeling_impulsive_need_advice/', 'reddit.com/r/BPD/comments/atbcln/im_in_love_with_someone_i_dont_know/', 'reddit.com/r/BPD/comments/5ega6h/my_bpd_so_is_scaring_me/', 'reddit.com/r/BPD/comments/a1fv13/bpd_ex_and_struggle/']


In [11]:
def api_to_json(api):
  response = urlopen(api)
  return json.loads(response.read())

In [77]:
def get_char(num=0):
  import string
  from itertools import combinations_with_replacement as cwr
  output = ["".join(comb) for comb in cwr(string.ascii_lowercase, 2)][num]
  return output

def json_to_bibtex(json,num):
  json = json_data['data']['children'][0]['data']
  author = json['author']
  author_clean = author.replace('[','').replace(']','')
  day = datetime.utcfromtimestamp(json['created']).strftime('%d')
  month = datetime.utcfromtimestamp(json['created']).strftime('%b').lower()
  year = datetime.utcfromtimestamp(json['created']).strftime('%Y')
  accessed = datetime.now().strftime('%Y-%m-%d')
  title = json['title']
  site = json['url']
  forum = "reddit.com/r/"+json['subreddit']
#  abstract = json['selftext'].replace("\n","")
#  abstract = (abstract if len(abstract)<=200 else abstract[:200]+"...")
  return f"""@MISC{{{author_clean}{str(datetime.utcfromtimestamp(json['created']).strftime('%Y'))}-{get_char(num)},
          title\t\t= "{title}",
          booktitle\t= "{forum}",
          author\t= "{{{author}}}",
          month\t\t= {month},
          year\t\t= {year},
          day\t\t= {day},
          howpublished\t= "\\url{{{site}}}",
          note\t\t= "Accessed: {accessed}"
}}\n"""

@MISC{deleted2018-aa,
          title		= "Bpd ex and struggle",
          booktitle	= "reddit.com/r/BPD",
          author	= "{[deleted]}",
          month		= nov,
          year		= 2018,
          day		= 29,
          howpublished	= "\url{https://www.reddit.com/r/BPD/comments/a1fv13/bpd_ex_and_struggle/}",
          note		= "Accessed: 2022-02-08"
}



In [75]:
bibtex=""
#for i,url2 in enumerate(urls):
for idx,url in tqdm(enumerate(urls)):
  print(url)
  while True:
    try:
      api = url_to_api(url)
      json_data = api_to_json(api)
      bibtex+=json_to_bibtex(json_data, idx)
      break
    except urllib.error.HTTPError:
      print("Rate limited. Sleeping one minute")
      for i in tqdm(range(60)):
        sleep(1)
      continue

#print("\n\n------- it looks like this -----\n"+bibtex[0])
with open('reddit.bib', 'w') as f:
    f.write(bibtex)
print("\n\n\n... Saved file to reddit.bib")


found urls:


0it [00:00, ?it/s]

reddit.com/r/BPD/comments/7x3jv4/i_feel_like_the_universe_is_asking_me_to_die/
Rate limited. Sleeping one minute


  0%|          | 0/60 [00:00<?, ?it/s]

Rate limited. Sleeping one minute


...

Rate limited. Sleeping one minute


  0%|          | 0/60 [00:00<?, ?it/s]

reddit.com/r/BPD/comments/9o804o/just_looking_for_some_advice_with_paranoia/
reddit.com/r/BPD/comments/4z7bks/in_a_new_relationship/
reddit.com/r/BPD/comments/68ighc/feeling_impulsive_need_advice/
reddit.com/r/BPD/comments/atbcln/im_in_love_with_someone_i_dont_know/
Rate limited. Sleeping one minute


  0%|          | 0/60 [00:00<?, ?it/s]

reddit.com/r/BPD/comments/5ega6h/my_bpd_so_is_scaring_me/
reddit.com/r/BPD/comments/a1fv13/bpd_ex_and_struggle/
@MISC{GoneUnsane2018-aa,
          title	="I feel like the universe is asking me to die",
          booktitle	= "BPD",
          author	= "{GoneUnsane}",
          abstract	= "I feel like I have always had BPD, I've still got vivid memories of elementary school classes, and recess. Neither were ever enjoyable to me, I'd spend most of the time stargazing why life happen to m...",
          month		= feb,
          year		= 2018,
          day		= 12,
          howpublished	= "\url{https://www.reddit.com/r/BPD/comments/7x3jv4/i_feel_like_the_universe_is_asking_me_to_die/}",
          note		= "Accessed: 2022-02-08"
}
@MISC{wendypan1052019-ab,
          title	="Paranoia",
          booktitle	= "BPD",
          author	= "{wendypan105}",
          abstract	= "I've been with my partner 7 nearly 8 months and while he hasn't  shown any reason to distrust him  I am  just so paranoid I am 